<a href="https://colab.research.google.com/github/djg230/jupyter-notebooks/blob/main/AttackComparison.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip3 install foolbox==3.1.1
# !pip3 install git+https://github.com/bethgelab/foolbox.git

In [ ]:
import foolbox as fb
import foolbox.attacks as fa

In [ ]:

import eagerpy as ep
from foolbox import PyTorchModel, accuracy, samples
import foolbox.attacks as fa
import numpy as np
import matplotlib.pyplot as plt
import datetime

In [ ]:
import torch
import torchvision

In [ ]:
torch.__version__

'1.8.0+cu101'

In [ ]:
torch.cuda.is_available()

False


# Import resnet18 model and preprocess

In [ ]:
model = torchvision.models.resnet18(pretrained=True)

In [ ]:
model = model.eval()

In [ ]:
# PyTorch ResNet18
preprocessing = dict(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225], axis=-3)
bounds = (0, 1)
fmodel = fb.PyTorchModel(model, bounds=bounds, preprocessing=preprocessing)

In [ ]:
fmodel = fmodel.transform_bounds((0, 1))
assert fmodel.bounds == (0, 1)

In [ ]:
images, labels = fb.utils.samples(fmodel, dataset='imagenet', batchsize=16)
images = ep.astensor(images)
labels = ep.astensor(labels)

#The list of attacks being used. 

The same epsilon of 0.5 is used for all attacks. I found that, since im attacking 16 images in each attack, using a range of epsilons caused each individual attack to take an incredibly long time, which is why i settled on attacking using one epsilon. Attacks that had a default step amount of over 100 had their steps set to 100.

In [ ]:
#epsilons = np.linspace(0.0, 1, num=10)
epsilons = 0.5
attacks = [
        fa.L2CarliniWagnerAttack(steps = 100),
        fa.L2ContrastReductionAttack(),
        fa.DDNAttack(),
        fa.L2BasicIterativeAttack(),
        fa.LinfBasicIterativeAttack(),
        fa.L2AdditiveGaussianNoiseAttack(),
        fa.SaltAndPepperNoiseAttack(steps = 100),
        fa.NewtonFoolAttack(),
        fa.L2ClippingAwareAdditiveGaussianNoiseAttack(),
        fa.L2AdditiveUniformNoiseAttack()
]

Running all of the attacks at once. I also listed each test individually, to test each one isolated.

In [ ]:
  for attack in attacks:
    start = datetime.datetime.now()
    advs, _, is_adv = attack(fmodel, images, labels, epsilons=epsilons)
    end = datetime.datetime.now()
    print("======ATTACK NAME: {} ======".format(attack))
    print("attack success rate:")
    attack_success_rate = is_adv.float32().mean(axis=-1)
    print(attack_success_rate)
    print("")
    print("L0 norms:")
    print(fb.distances.l0(images, advs))
    print("")
    print("L1 norms:")
    print(fb.distances.l1(images, advs))
    print("")
    print("L2 norms:")
    print(fb.distances.l2(images, advs))
    print("")
    print("Linf norms:")
    print(fb.distances.linf(images, advs))
    print("")
    print('Time cost: {} seconds'.format(((end-start).total_seconds())))

======ATTACK NAME: L2CarliniWagnerAttack(binary_search_steps=9, steps=100, stepsize=0.01, confidence=0, initial_const=0.001, abort_early=True) ======
attack success rate:
PyTorchTensor(tensor(0.7500))

L0 norms:
PyTorchTensor(tensor([150528., 150528., 150527., 150528., 150527., 150527., 150525., 145556.,
                      150527., 150528., 150528., 150521., 150528., 150528., 150528., 150528.]))

L1 norms:
PyTorchTensor(tensor([1.1014e+02, 2.2434e+02, 9.4654e+01, 9.2975e+01, 1.5225e+02, 1.1409e+02,
                      7.0907e+01, 3.3154e-02, 1.8689e+02, 2.2659e+02, 1.0745e+02, 8.8289e+01,
                      2.0281e+02, 1.1860e+02, 7.7198e+01, 2.1298e+02]))

L2 norms:
PyTorchTensor(tensor([3.2728e-01, 6.3755e-01, 3.0770e-01, 2.9790e-01, 4.0746e-01, 3.4634e-01,
                      2.4577e-01, 9.5528e-05, 6.1016e-01, 6.4899e-01, 4.0220e-01, 2.7661e-01,
                      5.4893e-01, 3.5090e-01, 2.6737e-01, 5.6474e-01]))

Linf norms:
PyTorchTensor(tensor([3.7018e-03, 2.3508e-0

All attacks individually:

In [ ]:
  attack = fb.attacks.L2CarliniWagnerAttack(steps=100)
  start = datetime.datetime.now()
  advs, _, is_adv = attack(fmodel, images, labels, epsilons=epsilons)
  end = datetime.datetime.now()
  print("======ATTACK NAME: L2 Carlini Wagner Attack ======")
  print("attack success rate:")
  attack_success_rate = is_adv.float32().mean(axis=-1)
  print(attack_success_rate)
  print("")
  print("L0 norms:")
  print(fb.distances.l0(images, advs))
  print("")
  print("L1 norms:")
  print(fb.distances.l1(images, advs))
  print("")
  print("L2 norms:")
  print(fb.distances.l2(images, advs))
  print("")
  print("Linf norms:")
  print(fb.distances.linf(images, advs))
  print("")
  print('Time cost: {} seconds'.format(((end-start).total_seconds())))

======ATTACK NAME: L2 Carlini Wagner Attack ======
attack success rate:
PyTorchTensor(tensor(0.7500))

L0 norms:
PyTorchTensor(tensor([150528., 150528., 150527., 150528., 150527., 150527., 150525., 145556.,
                      150527., 150528., 150528., 150521., 150528., 150528., 150528., 150528.]))

L1 norms:
PyTorchTensor(tensor([1.1014e+02, 2.2434e+02, 9.4654e+01, 9.2975e+01, 1.5225e+02, 1.1409e+02,
                      7.0907e+01, 3.3154e-02, 1.8689e+02, 2.2659e+02, 1.0745e+02, 8.8289e+01,
                      2.0281e+02, 1.1860e+02, 7.7198e+01, 2.1298e+02]))

L2 norms:
PyTorchTensor(tensor([3.2728e-01, 6.3755e-01, 3.0770e-01, 2.9790e-01, 4.0746e-01, 3.4634e-01,
                      2.4577e-01, 9.5528e-05, 6.1016e-01, 6.4899e-01, 4.0220e-01, 2.7661e-01,
                      5.4893e-01, 3.5090e-01, 2.6737e-01, 5.6474e-01]))

Linf norms:
PyTorchTensor(tensor([3.7018e-03, 2.3508e-03, 3.7060e-03, 3.7045e-03, 3.5225e-03, 3.6988e-03,
                      2.0804e-03, 5.3644e-07, 2.

In [ ]:
  attack = fa.L2ContrastReductionAttack()
  start = datetime.datetime.now()
  advs, _, is_adv = attack(fmodel, images, labels, epsilons=epsilons)
  end = datetime.datetime.now()
  print("======ATTACK NAME: L2 Contrast Reduction Attack ======")
  print("attack success rate:")
  attack_success_rate = is_adv.float32().mean(axis=-1)
  print(attack_success_rate)
  print("")
  print("L0 norms:")
  print(fb.distances.l0(images, advs))
  print("")
  print("L1 norms:")
  print(fb.distances.l1(images, advs))
  print("")
  print("L2 norms:")
  print(fb.distances.l2(images, advs))
  print("")
  print("Linf norms:")
  print(fb.distances.linf(images, advs))
  print("")
  print('Time cost: {} seconds'.format(((end-start).total_seconds())))

======ATTACK NAME: L2 Contrast Reduction Attack ======
attack success rate:
PyTorchTensor(tensor(0.0625))

L0 norms:
PyTorchTensor(tensor([150528., 150528., 150528., 150528., 150528., 150528., 150528., 150528.,
                      150528., 150528., 150528., 150528., 150528., 150528., 150528., 150528.]))

L1 norms:
PyTorchTensor(tensor([166.8194, 159.6962, 173.1231, 168.9075, 163.1219, 154.4686, 161.1778,
                      173.2364, 173.8341, 175.0330, 164.9120, 178.4924, 161.4643, 154.1938,
                      142.8475, 163.1408]))

L2 norms:
PyTorchTensor(tensor([0.5000, 0.5000, 0.5000, 0.5000, 0.5000, 0.5000, 0.5000, 0.5000, 0.5000,
                      0.5000, 0.5000, 0.5000, 0.5000, 0.5000, 0.5000, 0.5000]))

Linf norms:
PyTorchTensor(tensor([0.0028, 0.0027, 0.0027, 0.0024, 0.0029, 0.0031, 0.0026, 0.0026, 0.0020,
                      0.0023, 0.0024, 0.0019, 0.0027, 0.0040, 0.0048, 0.0031]))

Time cost: 1.619489 seconds


In [ ]:
  attack = fa.DDNAttack()
  start = datetime.datetime.now()
  advs, _, is_adv = attack(fmodel, images, labels, epsilons=epsilons)
  end = datetime.datetime.now()
  print("======ATTACK NAME: DDNAttack ======")
  print("attack success rate:")
  attack_success_rate = is_adv.float32().mean(axis=-1)
  print(attack_success_rate)
  print("")
  print("L0 norms:")
  print(fb.distances.l0(images, advs))
  print("")
  print("L1 norms:")
  print(fb.distances.l1(images, advs))
  print("")
  print("L2 norms:")
  print(fb.distances.l2(images, advs))
  print("")
  print("Linf norms:")
  print(fb.distances.linf(images, advs))
  print("")
  print('Time cost: {} seconds'.format(((end-start).total_seconds())))

======ATTACK NAME: DDNAttack ======
attack success rate:
PyTorchTensor(tensor(0.9375))

L0 norms:
PyTorchTensor(tensor([150447., 150521., 150436., 150462., 150467., 150230., 150524.,      0.,
                      147010., 150527., 150251., 145195., 149742., 150524., 150510., 150520.]))

L1 norms:
PyTorchTensor(tensor([160.7966, 189.6486, 189.9163, 181.6256, 199.2537, 175.9630, 173.5249,
                        0.0000, 181.2720, 196.4261, 191.1758, 176.8514, 190.9931, 191.1130,
                      167.2402, 178.2068]))

L2 norms:
PyTorchTensor(tensor([0.6966, 0.6966, 0.6966, 0.6966, 0.6966, 0.6966, 0.6966, 0.0000, 0.7699,
                      0.6966, 0.6966, 0.6966, 0.6966, 0.6966, 0.6966, 0.6966]))

Linf norms:
PyTorchTensor(tensor([0.0269, 0.0215, 0.0226, 0.0272, 0.0137, 0.0208, 0.0432, 0.0000, 0.0276,
                      0.0172, 0.0185, 0.0298, 0.0223, 0.0164, 0.0322, 0.0240]))

Time cost: 48.730274 seconds


In [ ]:
  attack = fa.L2BasicIterativeAttack()
  start = datetime.datetime.now()
  advs, _, is_adv = attack(fmodel, images, labels, epsilons=epsilons)
  end = datetime.datetime.now()
  print("======ATTACK NAME: L2 Basic Iterative Attack ======")
  print("attack success rate:")
  attack_success_rate = is_adv.float32().mean(axis=-1)
  print(attack_success_rate)
  print("")
  print("L0 norms:")
  print(fb.distances.l0(images, advs))
  print("")
  print("L1 norms:")
  print(fb.distances.l1(images, advs))
  print("")
  print("L2 norms:")
  print(fb.distances.l2(images, advs))
  print("")
  print("Linf norms:")
  print(fb.distances.linf(images, advs))
  print("")
  print('Time cost: {} seconds'.format(((end-start).total_seconds())))

======ATTACK NAME: L2 Basic Iterative Attack ======
attack success rate:
PyTorchTensor(tensor(1.))

L0 norms:
PyTorchTensor(tensor([150439., 150518., 150409., 150466., 150442., 150203., 150518., 150468.,
                      146521., 150524., 150252., 142794., 149722., 150522., 150505., 150524.]))

L1 norms:
PyTorchTensor(tensor([110.7135, 137.1846, 137.8759, 126.3066, 139.6180, 119.2714, 126.9673,
                      139.2497, 112.5871, 138.8176, 137.2085, 128.2836, 136.5885, 140.1088,
                      121.4503, 124.6061]))

L2 norms:
PyTorchTensor(tensor([0.5000, 0.5000, 0.5000, 0.5000, 0.5000, 0.5000, 0.5000, 0.5000, 0.5000,
                      0.5000, 0.5000, 0.4997, 0.5000, 0.5000, 0.5000, 0.5000]))

Linf norms:
PyTorchTensor(tensor([0.0252, 0.0163, 0.0137, 0.0237, 0.0129, 0.0236, 0.0179, 0.0137, 0.0191,
                      0.0153, 0.0138, 0.0278, 0.0150, 0.0127, 0.0200, 0.0233]))

Time cost: 48.642005 seconds


In [ ]:
  attack = fa.LinfBasicIterativeAttack()
  start = datetime.datetime.now()
  advs, _, is_adv = attack(fmodel, images, labels, epsilons=epsilons)
  end = datetime.datetime.now()
  print("======ATTACK NAME: Linf Basic Iterative Attack ======")
  print("attack success rate:")
  attack_success_rate = is_adv.float32().mean(axis=-1)
  print(attack_success_rate)
  print("")
  print("L0 norms:")
  print(fb.distances.l0(images, advs))
  print("")
  print("L1 norms:")
  print(fb.distances.l1(images, advs))
  print("")
  print("L2 norms:")
  print(fb.distances.l2(images, advs))
  print("")
  print("Linf norms:")
  print(fb.distances.linf(images, advs))
  print("")
  print('Time cost: {} seconds'.format(((end-start).total_seconds())))

======ATTACK NAME: Linf Basic Iterative Attack ======
attack success rate:
PyTorchTensor(tensor(1.))

L0 norms:
PyTorchTensor(tensor([127769., 126826., 124374., 130787., 121805., 123361., 126422., 132567.,
                      134565., 128039., 126745., 127706., 127675., 120902., 114924., 124309.]))

L1 norms:
PyTorchTensor(tensor([28957.6719, 29583.4648, 28440.8301, 28092.3027, 29234.4336, 30121.9238,
                      29055.1836, 28676.5820, 27650.2734, 27663.5410, 28834.8711, 27578.3477,
                      29066.8301, 30981.5684, 31232.7070, 29364.8652]))

L2 norms:
PyTorchTensor(tensor([ 93.9197,  95.7196,  92.4944,  92.0553,  95.0104,  97.1819,  94.5973,
                       93.0718,  91.3042,  90.8652,  93.9911,  91.2042,  94.3984,  99.2548,
                      100.1334,  94.9984]))

Linf norms:
PyTorchTensor(tensor([0.5000, 0.5000, 0.5000, 0.5000, 0.5000, 0.5000, 0.5000, 0.5000, 0.5000,
                      0.5000, 0.5000, 0.5000, 0.5000, 0.5000, 0.5000, 0.5000]))



In [ ]:
  attack = fa.L2AdditiveGaussianNoiseAttack()
  start = datetime.datetime.now()
  advs, _, is_adv = attack(fmodel, images, labels, epsilons=epsilons)
  end = datetime.datetime.now()
  print("======ATTACK NAME: L2 Additive Gaussian Noise Attack ======")
  print("attack success rate:")
  attack_success_rate = is_adv.float32().mean(axis=-1)
  print(attack_success_rate)
  print("")
  print("L0 norms:")
  print(fb.distances.l0(images, advs))
  print("")
  print("L1 norms:")
  print(fb.distances.l1(images, advs))
  print("")
  print("L2 norms:")
  print(fb.distances.l2(images, advs))
  print("")
  print("Linf norms:")
  print(fb.distances.linf(images, advs))
  print("")
  print('Time cost: {} seconds'.format(((end-start).total_seconds())))

======ATTACK NAME: Linf Basic Iterative Attack ======
attack success rate:
PyTorchTensor(tensor(0.0625))

L0 norms:
PyTorchTensor(tensor([150446., 150520., 150361., 150448., 150418., 150170., 150525., 150447.,
                      145743., 150526., 150219., 136840., 149726., 150522., 150495., 150526.]))

L1 norms:
PyTorchTensor(tensor([154.7746, 154.7046, 154.4294, 154.7014, 154.5184, 154.3954, 154.9389,
                      154.7644, 149.6208, 154.7926, 154.3581, 140.6115, 153.9662, 154.8448,
                      154.6798, 154.7283]))

L2 norms:
PyTorchTensor(tensor([0.4999, 0.5000, 0.4997, 0.4999, 0.4998, 0.4994, 0.5000, 0.4999, 0.4918,
                      0.5000, 0.4995, 0.4767, 0.4987, 0.5000, 0.5000, 0.5000]))

Linf norms:
PyTorchTensor(tensor([0.0058, 0.0060, 0.0063, 0.0063, 0.0061, 0.0062, 0.0061, 0.0062, 0.0058,
                      0.0058, 0.0062, 0.0058, 0.0063, 0.0061, 0.0058, 0.0062]))

Time cost: 1.389469 seconds


In [ ]:
  attack = fa.SaltAndPepperNoiseAttack()
  start = datetime.datetime.now()
  advs, _, is_adv = attack(fmodel, images, labels, epsilons=epsilons)
  end = datetime.datetime.now()
  print("======ATTACK NAME: L2 Additive Gaussian Noise Attack ======")
  print("attack success rate:")
  attack_success_rate = is_adv.float32().mean(axis=-1)
  print(attack_success_rate)
  print("")
  print("L0 norms:")
  print(fb.distances.l0(images, advs))
  print("")
  print("L1 norms:")
  print(fb.distances.l1(images, advs))
  print("")
  print("L2 norms:")
  print(fb.distances.l2(images, advs))
  print("")
  print("Linf norms:")
  print(fb.distances.linf(images, advs))
  print("")
  print('Time cost: {} seconds'.format(((end-start).total_seconds())))

======ATTACK NAME: L2 Additive Gaussian Noise Attack ======
attack success rate:
PyTorchTensor(tensor(0.0625))

L0 norms:
PyTorchTensor(tensor([1.5000e+02, 1.0820e+03, 2.9900e+02, 7.4400e+02, 1.5471e+04, 1.1700e+02,
                      3.9000e+01, 0.0000e+00, 1.3740e+03, 3.8037e+04, 4.1930e+03, 1.8600e+02,
                      3.1429e+04, 1.0710e+03, 1.2000e+01, 2.2230e+03]))

L1 norms:
PyTorchTensor(tensor([7.3663e+01, 5.3639e+02, 1.5471e+02, 3.5802e+02, 7.7686e+03, 6.1086e+01,
                      2.6541e+01, 0.0000e+00, 7.1110e+02, 1.9006e+04, 2.1305e+03, 1.1451e+02,
                      1.5725e+04, 5.4318e+02, 4.4745e+00, 1.0865e+03]))

L2 norms:
PyTorchTensor(tensor([  6.6280,  18.0658,   9.8822,  14.9698,  68.4303,   6.0155,   4.4386,
                        0.0000,  22.3669, 111.3775,  36.9717,   9.5015,  97.8181,  17.3733,
                        1.4589,  25.0893]))

Linf norms:
PyTorchTensor(tensor([0.9412, 0.9922, 0.9843, 0.9961, 1.0000, 0.9529, 0.9647, 0.0000, 1.0000,
 

In [ ]:
  attack = fa.NewtonFoolAttack()
  start = datetime.datetime.now()
  advs, _, is_adv = attack(fmodel, images, labels, epsilons=epsilons)
  end = datetime.datetime.now()
  print("======ATTACK NAME: Newton Fool Attack ======")
  print("attack success rate:")
  attack_success_rate = is_adv.float32().mean(axis=-1)
  print(attack_success_rate)
  print("")
  print("L0 norms:")
  print(fb.distances.l0(images, advs))
  print("")
  print("L1 norms:")
  print(fb.distances.l1(images, advs))
  print("")
  print("L2 norms:")
  print(fb.distances.l2(images, advs))
  print("")
  print("Linf norms:")
  print(fb.distances.linf(images, advs))
  print("")
  print('Time cost: {} seconds'.format(((end-start).total_seconds())))

======ATTACK NAME: L2 Additive Gaussian Noise Attack ======
attack success rate:
PyTorchTensor(tensor(0.6875))

L0 norms:
PyTorchTensor(tensor([150411., 150516., 150387., 150456., 150429., 150144., 150483.,      0.,
                      145769., 150526., 150229., 137026., 149667., 150521., 150456., 150526.]))

L1 norms:
PyTorchTensor(tensor([2.2077e+01, 4.6774e+04, 3.7900e+01, 4.1146e+01, 8.2313e+03, 1.3383e+04,
                      9.5795e+00, 0.0000e+00, 7.5173e+04, 7.3852e+04, 7.4473e+04, 3.3269e+02,
                      1.0137e+02, 3.0762e+02, 1.1231e+01, 5.4563e+04]))

L2 norms:
PyTorchTensor(tensor([1.0502e-01, 1.4844e+02, 1.4212e-01, 1.6122e-01, 2.9372e+01, 5.6520e+01,
                      3.9232e-02, 0.0000e+00, 2.2967e+02, 2.1837e+02, 2.1764e+02, 1.3587e+00,
                      3.8557e-01, 1.1129e+00, 4.6531e-02, 1.6537e+02]))

Linf norms:
PyTorchTensor(tensor([0.0049, 1.0000, 0.0063, 0.0061, 0.6823, 0.9412, 0.0019, 0.0000, 1.0000,
                      1.0000, 1.0000, 0

In [ ]:
  attack = fa.L2AdditiveUniformNoiseAttack()
  start = datetime.datetime.now()
  advs, _, is_adv = attack(fmodel, images, labels, epsilons=epsilons)
  end = datetime.datetime.now()
  print("======ATTACK NAME: L2 Additive Uniform Noise Attack ======")
  print("attack success rate:")
  attack_success_rate = is_adv.float32().mean(axis=-1)
  print(attack_success_rate)
  print("")
  print("L0 norms:")
  print(fb.distances.l0(images, advs))
  print("")
  print("L1 norms:")
  print(fb.distances.l1(images, advs))
  print("")
  print("L2 norms:")
  print(fb.distances.l2(images, advs))
  print("")
  print("Linf norms:")
  print(fb.distances.linf(images, advs))
  print("")
  print('Time cost: {} seconds'.format(((end-start).total_seconds())))

======ATTACK NAME: L2 Additive Uniform Noise Attack ======
attack success rate:
PyTorchTensor(tensor(0.0625))

L0 norms:
PyTorchTensor(tensor([150441., 150514., 150395., 150459., 150422., 150168., 150520., 150455.,
                      145730., 150524., 150234., 137191., 149740., 150519., 150500., 150525.]))

L1 norms:
PyTorchTensor(tensor([167.8417, 167.8664, 167.8806, 168.0382, 167.8939, 167.6119, 167.9347,
                      167.9128, 162.6051, 167.9453, 167.6131, 153.2142, 167.1090, 167.8979,
                      168.0092, 168.0011]))

L2 norms:
PyTorchTensor(tensor([0.4999, 0.5000, 0.4998, 0.4999, 0.4998, 0.4994, 0.5000, 0.4999, 0.4919,
                      0.5000, 0.4995, 0.4774, 0.4987, 0.5000, 0.5000, 0.5000]))

Linf norms:
PyTorchTensor(tensor([0.0022, 0.0022, 0.0022, 0.0022, 0.0022, 0.0022, 0.0022, 0.0022, 0.0022,
                      0.0022, 0.0022, 0.0022, 0.0022, 0.0022, 0.0022, 0.0022]))

Time cost: 1.334737 seconds


In [ ]:
  attack = fa.L2ClippingAwareAdditiveGaussianNoiseAttack()
  start = datetime.datetime.now()
  advs, _, is_adv = attack(fmodel, images, labels, epsilons=epsilons)
  end = datetime.datetime.now()
  print("======ATTACK NAME: L2 Clipping Aware Additive Gaussian Noise Attack ======")
  print("attack success rate:")
  attack_success_rate = is_adv.float32().mean(axis=-1)
  print(attack_success_rate)
  print("")
  print("L0 norms:")
  print(fb.distances.l0(images, advs))
  print("")
  print("L1 norms:")
  print(fb.distances.l1(images, advs))
  print("")
  print("L2 norms:")
  print(fb.distances.l2(images, advs))
  print("")
  print("Linf norms:")
  print(fb.distances.linf(images, advs))
  print("")
  print('Time cost: {} seconds'.format(((end-start).total_seconds())))

======ATTACK NAME: L2 Clipping Aware Additive Gaussian Noise Attack ======
attack success rate:
PyTorchTensor(tensor(0.0625))

L0 norms:
PyTorchTensor(tensor([150438., 150520., 150387., 150457., 150420., 150173., 150525., 150450.,
                      145710., 150524., 150218., 137063., 149756., 150522., 150502., 150526.]))

L1 norms:
PyTorchTensor(tensor([154.8324, 154.8331, 154.8308, 154.7361, 154.7878, 154.5051, 154.7570,
                      154.7657, 152.2947, 154.7318, 154.5631, 147.7436, 154.4474, 154.8196,
                      154.8004, 154.7869]))

L2 norms:
PyTorchTensor(tensor([0.5000, 0.5000, 0.5000, 0.5000, 0.5000, 0.5000, 0.5000, 0.5000, 0.5000,
                      0.5000, 0.5000, 0.5000, 0.5000, 0.5000, 0.5000, 0.5000]))

Linf norms:
PyTorchTensor(tensor([0.0062, 0.0058, 0.0055, 0.0059, 0.0056, 0.0055, 0.0066, 0.0056, 0.0059,
                      0.0056, 0.0056, 0.0070, 0.0061, 0.0074, 0.0060, 0.0058]))

Time cost: 1.900151 seconds
